#### Columnas Originales

* ITEM_ID: id unívoco de cada item publicado. (Ofuscado)
* SHP_WEIGHT: peso del paquete informado por el correo.
* SHP_LENGTH: largo del paquete informado por el correo.
* SHP_WIDTH: ancho del paquete informado por el correo.
* SHP_HEIGHT: altura del paquete informado por el correo.
* ATTRIBUTES: atributos como marca y modelo, entre otros, en formato json-lines
* CATALOG_PRODUCT_ID: id del catálogo (ofuscado).
* CONDITION: condición de venta (nuevo o usado).
* DOMAIN_ID: id de la categoría a la que pertenece la publicación.
* PRICE: precio en reales.
* SELLER_ID: id del vendedor (ofuscado).
* STATUS: estado de la publicación (activa, cerrada, pausada, etc.)
* TITLE: título de la publicación.


#### Columnas Actuales

* ITEM_ID
* SHP_WEIGHT
* SHP_LENGTH
* SHP_WIDTH
* SHP_HEIGHT
* PRICE
* STATUS
* TITLE
* LEN_ATR: cantidad de atributos
* DT_CAT_PROD: ID Catalogo del Producto-Revisado
* DT_CONDITION: Condición de Venta -Revisado
* DT_DOMAIN: Categoría de la Publicación -Revisado
* DT_SELLER: ID Vendedor -Revisado
* DT_BRAND: Marca del Producto -Revisado
* DT_MODEL: Modelo del Producto -Revisado
* SCORE: Peso ponderado del Título de la publicación sobre si excede la dimensión máxima
* EXCEDIDO: Si el producto excede el límite del correo


In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import ast

from itertools import chain
from collections import Counter
from pandas.io.json import json_normalize
from sklearn import preprocessing

import random
random.seed(0)

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
DATASET = 'dataset/meli_dataset_b.csv'

In [3]:
df_raw = pd.read_csv(DATASET, low_memory=False)
df_base = df_raw.copy()

In [4]:
print(df_base.shape)
print(df_base.dtypes)
display(df_base.sample(5))

(296291, 17)
ITEM_ID          object
SHP_WEIGHT      float64
SHP_LENGTH      float64
SHP_WIDTH       float64
SHP_HEIGHT      float64
PRICE           float64
STATUS           object
TITLE            object
LEN_ATR           int64
DT_CAT_PROD      object
DT_CONDITION     object
DT_DOMAIN        object
DT_SELLER        object
DT_BRAND         object
DT_MODEL         object
SCORE           float64
EXCEDIDO          int64
dtype: object


,ITEM_ID,SHP_WEIGHT,SHP_LENGTH,SHP_WIDTH,SHP_HEIGHT,PRICE,STATUS,TITLE,LEN_ATR,DT_CAT_PROD,DT_CONDITION,DT_DOMAIN,DT_SELLER,DT_BRAND,DT_MODEL,SCORE,EXCEDIDO
238590,DJZI01BYSU,315.0,18.0,16.0,8.0,39.5,active,Pressostato Lavadora Colormaq Lca 12kg Lca 15kg,9,H53U1H7Q5G,new,MLB-HOME_APPLIANCES_ACCESSORIES_AND_SPARE_PARTS,XP959BRI59,COLORMAQ,OTROS,0.000329,0
120343,F8BP5FI083,1220.0,20.0,19.0,18.0,219.0,active,Secador De Cabelo Taiff Rs5 1900w 110volts,14,OTROS,new,MLB-HAIR_DRYERS,F0FOFA3V9E,TAIFF,OTROS,0.000631,0
174244,B6CBVJSMPF,440.0,19.0,12.0,8.0,320.0,paused,Hd Externo 1tb Portátil Seagate Expansion 2.5 ...,19,YZDEE9O9Q2,new,MLB-HARD_DRIVES_AND_SSDS,OTROS,SEAGATE,OTROS,0.000057,0
263660,PR5N7848GY,235.0,25.0,25.0,5.0,39.9,active,Cinta Modeladora Redu Shaper Ação Térmica Femi...,6,H53U1H7Q5G,new,MLB-BODY_SHAPERS,SM0LRRNBM6,IMPORTADO,OTROS,0.000044,0
130461,XXY9J9EA1F,5720.0,64.0,41.0,32.0,158.0,active,Par Lente Farol Honda Fit Ano 2003 2004 2005 2...,4,H53U1H7Q5G,new,MLB-VEHICLE_PARTS,OTROS,IMPORTADO,OTROS,0.770420,0


In [5]:
df_base.describe()

,SHP_WEIGHT,SHP_LENGTH,SHP_WIDTH,SHP_HEIGHT,PRICE,LEN_ATR,SCORE,EXCEDIDO
count,296291.000000,296291.000000,296291.000000,296291.000000,261924.000000,296291.000000,296291.000000,296291.000000
mean,1861.161446,31.426163,21.318484,11.562582,176.781718,7.918637,0.122348,0.044925
std,3254.785995,18.619454,11.301392,8.358128,444.476057,5.676726,0.261096,0.207141
min,1.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000014,0.000000
25%,253.000000,20.000000,13.000000,5.000000,48.900000,4.000000,0.000282,0.000000
50%,680.000000,25.000000,20.000000,10.000000,100.000000,8.000000,0.002085,0.000000
75%,1948.000000,36.000000,25.000000,16.000000,179.000000,11.000000,0.055526,0.000000
max,50000.000000,288.200000,165.000000,105.000000,30000.000000,60.000000,0.999856,1.000000


In [6]:
df_base.describe(include=['O'])

,ITEM_ID,STATUS,TITLE,DT_CAT_PROD,DT_CONDITION,DT_DOMAIN,DT_SELLER,DT_BRAND,DT_MODEL
count,296291,296291,296291,296291,296291,296291,296291,296291,296291
unique,236413,5,228346,84,4,1092,1584,1114,498
top,EZ118KZWRZ,active,Maquininha Point Mini - A Máquina De Cartão Do...,H53U1H7Q5G,new,SIN_DATOS,OTROS,OTROS,OTROS
freq,161,161493,164,261235,260545,35052,194855,111705,157977


#Se verifican valores nulos

In [7]:
#Quedan valores nulos de PRICE
df_base.isna().sum()

ITEM_ID             0
SHP_WEIGHT          0
SHP_LENGTH          0
SHP_WIDTH           0
SHP_HEIGHT          0
PRICE           34367
STATUS              0
TITLE               0
LEN_ATR             0
DT_CAT_PROD         0
DT_CONDITION        0
DT_DOMAIN           0
DT_SELLER           0
DT_BRAND            0
DT_MODEL            0
SCORE               0
EXCEDIDO            0
dtype: int64

In [8]:
#Verifico variable objetivo
df_base.EXCEDIDO.value_counts()

0    282980
1     13311
Name: EXCEDIDO, dtype: int64

#### Hacemos un análisis de los valores nulos en PRECIO, esto se utilizará luego para armar una estrategia y completar estos datos

In [9]:
#Verificamos cuantos casos Excedidos hay SIN Precio
df_base[ df_base.PRICE.isna() ].EXCEDIDO.value_counts()

0    33601
1      766
Name: EXCEDIDO, dtype: int64

In [10]:
#Solo hay valores faltantes de Precio en Status y Seller 
#Los respaldo para limitar la cantidad de columnas en el modelo
sinPrecio=df_base.PRICE.isna()
print('Status ', df_base[ sinPrecio ].STATUS.value_counts().count())
print('Catalog ', df_base[ sinPrecio ].DT_CAT_PROD.value_counts().count())
print('Condition ', df_base[ sinPrecio ].DT_CONDITION.value_counts().count())
print('Domain ', df_base[ sinPrecio ].DT_DOMAIN.value_counts().count())
print('Brand ', df_base[ sinPrecio ].DT_BRAND.value_counts().count())
print('Model ', df_base[ sinPrecio ].DT_MODEL.value_counts().count())
print('Seller ', df_base[ sinPrecio ].DT_SELLER.value_counts().count())


Status  3
Catalog  1
Condition  1
Domain  1
Brand  1
Model  1
Seller  499


In [11]:
dfSeller= df_base[ df_base.PRICE.isna() ].DT_SELLER.value_counts().to_frame()
dfStatus= df_base[ df_base.PRICE.isna() ].STATUS.value_counts().to_frame()

print(dfSeller.shape)
display(dfSeller.head(5))
print(dfStatus.shape)
display(dfStatus.head())


(499, 1)


,DT_SELLER
OTROS,22953
W3PZI7S738,493
GA9FI6X2KH,468
MRRLYZSZ5W,271
JQFLLZ2TN7,252


(3, 1)


,STATUS
under_review,31159
inactive,3026
closed,182


5. Transformar variables categóricas en números (Se recomienda OneHotEncoding) para las columnas (Sugerencia: arrancar con un sample de ~10K items)
`CATALOG_PRODUCT_ID`
`CONDITION`
`DOMAIN_ID`
`SELLER_ID`
`BRAND` (extraída en 4)
`MODEL`(extraída en 4)


In [12]:
col_categorias=['STATUS','DT_CAT_PROD','DT_CONDITION','DT_DOMAIN','DT_SELLER','DT_BRAND','DT_MODEL']

In [13]:
df=pd.get_dummies(df_base,columns=col_categorias, sparse=True)

In [14]:
df[[x for x in df.columns if 'STATUS' in x]].head(5)

,STATUS_active,STATUS_closed,STATUS_inactive,STATUS_paused,STATUS_under_review
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0


In [15]:
df.dtypes

ITEM_ID                      object
SHP_WEIGHT                  float64
SHP_LENGTH                  float64
SHP_WIDTH                   float64
SHP_HEIGHT                  float64
                         ...       
DT_MODEL_XENON     Sparse[uint8, 0]
DT_MODEL_XIAOMI    Sparse[uint8, 0]
DT_MODEL_XTREME    Sparse[uint8, 0]
DT_MODEL_X_PLAY    Sparse[uint8, 0]
DT_MODEL__         Sparse[uint8, 0]
Length: 4391, dtype: object

In [16]:
df.shape

(296291, 4391)

6. En caso de tener alguna variable no medida (en nuestro caso `PRICE`) imputar sus valores utilizando kNN.

In [17]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [18]:
#Vamos a completar la variable PRECIO utilizando kNN. Limitamos las columnas a aquellas con datos 
#para los casos nulos, revisado al principo de la notebook.
cols_size=['SHP_WEIGHT','SHP_LENGTH','SHP_WIDTH','SHP_HEIGHT',] 
cols_atr = ['STATUS_'+x for x in dfStatus.index]
cols_atr += ['DT_SELLER_'+x for x in dfSeller.index]

cols_knn=cols_size+cols_atr
print(len(cols_size))
print(len(cols_atr))
print(len(cols_knn))

4
502
506


* Separamos los envíos con PRECIO válido, este será nuestro X de base para entrenar y validar el modelo

In [19]:
dataset=df[ df.PRICE.notna() ].copy()

print(df.shape)
print(dataset.shape)

X=dataset[cols_knn]
y=dataset.PRICE

del dataset

print(X.shape)
print(y.shape)

(296291, 4391)
(261924, 4391)
(261924, 506)
(261924,)


In [20]:
mm=X.isnull().sum().to_frame()

In [21]:
mm.columns=['CANT']
mm[ mm.CANT >0 ]

,CANT


In [22]:
#Normalizo valores
scaler = MinMaxScaler()

display(X[ [x for x in X.columns if 'SHP_' in x]].head(5))

Xs = scaler.fit_transform(X)
X = pd.DataFrame(Xs, columns=X.columns)

display(X[ [x for x in X.columns if 'SHP_' in x]].head(5))
display(X[ [x for x in X.columns if 'SHP_' in x]].describe())


,SHP_WEIGHT,SHP_LENGTH,SHP_WIDTH,SHP_HEIGHT
0,235.0,25.0,25.0,10.0
1,1757.0,23.0,17.0,16.0
2,1748.0,23.0,17.0,16.0
3,1757.0,23.0,17.0,16.0
4,1780.0,23.0,17.0,16.0


,SHP_WEIGHT,SHP_LENGTH,SHP_WIDTH,SHP_HEIGHT
0,0.004680,0.086745,0.151515,0.095238
1,0.035121,0.079806,0.103030,0.152381
2,0.034941,0.079806,0.103030,0.152381
3,0.035121,0.079806,0.103030,0.152381
4,0.035581,0.079806,0.103030,0.152381


,SHP_WEIGHT,SHP_LENGTH,SHP_WIDTH,SHP_HEIGHT
count,261924.000000,261924.000000,261924.000000,261924.000000
mean,0.038424,0.110717,0.130932,0.112112
std,0.066110,0.066035,0.069575,0.080554
min,0.000000,0.000000,0.000000,0.000000
25%,0.005180,0.069396,0.078788,0.047619
50%,0.014180,0.086745,0.121212,0.095238
75%,0.040881,0.128383,0.153333,0.152381
max,1.000000,1.000000,1.000000,1.000000


In [23]:
print(X.shape)

(261924, 506)


In [24]:
#Separamos en train(80%) y test (20%) y entrenamos el modelo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [25]:
from sklearn.model_selection import GridSearchCV

In [26]:
#Buscamos mejores parámetros
params = {'n_neighbors':[3,5,7,9], 'metric':['manhattan','euclidean','hamming']}

mod_kn = KNeighborsRegressor(n_jobs=-1)
mod_GC  = GridSearchCV(mod_kn, params, cv=5)
mod_GC.fit(X_train,y_train)
mod_GC.best_params_

{'metric': 'manhattan', 'n_neighbors': 9}

In [27]:
for param, mean, std in zip(mod_GC.cv_results_['params'], 
                            mod_GC.cv_results_['mean_test_score'],
                            mod_GC.cv_results_['std_test_score']):
        print(param, mean, std)

{'metric': 'manhattan', 'n_neighbors': 3} 0.09380686409857274 0.024653824303686924
{'metric': 'manhattan', 'n_neighbors': 5} 0.17599725718138778 0.03707264482244945
{'metric': 'manhattan', 'n_neighbors': 7} 0.21754686404300905 0.04009363800621445
{'metric': 'manhattan', 'n_neighbors': 9} 0.23339881904931176 0.03815276541041238
{'metric': 'euclidean', 'n_neighbors': 3} 0.09221750733514704 0.02189064635763386
{'metric': 'euclidean', 'n_neighbors': 5} 0.18315111050851096 0.03604241363627244
{'metric': 'euclidean', 'n_neighbors': 7} 0.2173631459708647 0.04053391447480967
{'metric': 'euclidean', 'n_neighbors': 9} 0.2311752567848318 0.039757697141331705
{'metric': 'hamming', 'n_neighbors': 3} 0.0011295875943417835 0.022695340548001687
{'metric': 'hamming', 'n_neighbors': 5} 0.09332603700797804 0.016566159998051434
{'metric': 'hamming', 'n_neighbors': 7} 0.12245230582470815 0.020138770593887845
{'metric': 'hamming', 'n_neighbors': 9} 0.13643354266422136 0.020657679456846396


In [28]:
#Defino modelo con parámetros seleccionados
mod_nkk=KNeighborsRegressor(n_jobs=-1, **mod_GC.best_params_)

In [29]:
mod_nkk.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='manhattan',
                    metric_params=None, n_jobs=-1, n_neighbors=9, p=2,
                    weights='uniform')

In [30]:
#Verificamos la eficiencia del modelo en ambos conjuntos de trabajo
print('Train:', mod_nkk.score(X_train, y_train))
print('Train:', mod_nkk.score(X_test, y_test))

Train: 0.38000873613030517
Train: 0.17915480572639564


In [31]:
#Predicimos los PRECIOS para los envíos sin este dato
predict=df[ df.PRICE.isna() ].copy()
predict.shape

(34367, 4391)

In [32]:
predict.head(5)

,ITEM_ID,SHP_WEIGHT,SHP_LENGTH,SHP_WIDTH,SHP_HEIGHT,PRICE,TITLE,LEN_ATR,SCORE,EXCEDIDO,...,DT_MODEL_WS_887,DT_MODEL_X900,DT_MODEL_XBOX_360,DT_MODEL_XBOX_ONE,DT_MODEL_XBOX_ONE_S,DT_MODEL_XENON,DT_MODEL_XIAOMI,DT_MODEL_XTREME,DT_MODEL_X_PLAY,DT_MODEL__
11,S7TWYQJ0A4,180.0,25.0,11.0,5.0,NaN,Fone De Ouvido Airpods I7s Tws Iphone 5 6 7 8 ...,0,0.024387,0,...,0,0,0,0,0,0,0,0,0,0
23,L9JG6JZ286,1781.0,39.0,28.0,22.0,NaN,2 Kits E 6 Bolsas Pedido Especial De Cliente,0,0.007399,0,...,0,0,0,0,0,0,0,0,0,0
24,ZF8CEOICTA,722.0,25.0,15.0,8.0,NaN,Kit 2 Silver Scent Intenso Tester 100 Ml (fret...,0,0.000125,0,...,0,0,0,0,0,0,0,0,0,0
31,ANXV5SS0UL,151.0,16.0,16.0,16.0,NaN,Fidget Hand Spinner Fluoxetina,0,0.575820,0,...,0,0,0,0,0,0,0,0,0,0
33,BY1A1U3NXS,960.0,30.0,30.0,10.0,NaN,Caixa De Som Xtreme Mini Entrada Usb Pendrive...,0,0.089602,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
rPrice = mod_nkk.predict(scaler.transform(predict[cols_knn]))

In [34]:
rPrice=pd.DataFrame(data=rPrice,columns=['R_PRICE'])

In [35]:
print(rPrice.shape)
rPrice.head(5)

(34367, 1)


,R_PRICE
0,47.011111
1,115.658889
2,71.788889
3,42.531111
4,301.926667


In [36]:
predict.reset_index(inplace=True)
predict.head(5)

,index,ITEM_ID,SHP_WEIGHT,SHP_LENGTH,SHP_WIDTH,SHP_HEIGHT,PRICE,TITLE,LEN_ATR,SCORE,...,DT_MODEL_WS_887,DT_MODEL_X900,DT_MODEL_XBOX_360,DT_MODEL_XBOX_ONE,DT_MODEL_XBOX_ONE_S,DT_MODEL_XENON,DT_MODEL_XIAOMI,DT_MODEL_XTREME,DT_MODEL_X_PLAY,DT_MODEL__
0,11,S7TWYQJ0A4,180.0,25.0,11.0,5.0,NaN,Fone De Ouvido Airpods I7s Tws Iphone 5 6 7 8 ...,0,0.024387,...,0,0,0,0,0,0,0,0,0,0
1,23,L9JG6JZ286,1781.0,39.0,28.0,22.0,NaN,2 Kits E 6 Bolsas Pedido Especial De Cliente,0,0.007399,...,0,0,0,0,0,0,0,0,0,0
2,24,ZF8CEOICTA,722.0,25.0,15.0,8.0,NaN,Kit 2 Silver Scent Intenso Tester 100 Ml (fret...,0,0.000125,...,0,0,0,0,0,0,0,0,0,0
3,31,ANXV5SS0UL,151.0,16.0,16.0,16.0,NaN,Fidget Hand Spinner Fluoxetina,0,0.575820,...,0,0,0,0,0,0,0,0,0,0
4,33,BY1A1U3NXS,960.0,30.0,30.0,10.0,NaN,Caixa De Som Xtreme Mini Entrada Usb Pendrive...,0,0.089602,...,0,0,0,0,0,0,0,0,0,0


In [37]:
dfp=pd.merge(left=rPrice, right=predict['index'], how='left', left_index=True, right_index=True)
print(dfp.shape)
display(dfp.head(5)) 

(34367, 2)


,R_PRICE,index
0,47.011111,11
1,115.658889,23
2,71.788889,24
3,42.531111,31
4,301.926667,33


In [38]:
dfp.isna().sum()

R_PRICE    0
index      0
dtype: int64

In [39]:
dfp.index=dfp['index']
dfp.drop('index', axis=1, inplace=True)
print(dfp.shape)
display(dfp.head(5)) 

(34367, 1)


,R_PRICE
index,
11,47.011111
23,115.658889
24,71.788889
31,42.531111
33,301.926667


* Creamos una nueva columna con el PRECIO completo

In [40]:
df['RV_PRICE']=df.PRICE

In [41]:
df.RV_PRICE.fillna(dfp.R_PRICE, inplace=True)

In [42]:
df[ df.PRICE != df.RV_PRICE ].ITEM_ID.count()

34367

In [43]:
display(df[ df.PRICE.isna() ][['PRICE','RV_PRICE' ]].head(5))
display(df[ df.PRICE.isna() ][['PRICE','RV_PRICE' ]].describe())

,PRICE,RV_PRICE
11,NaN,47.011111
23,NaN,115.658889
24,NaN,71.788889
31,NaN,42.531111
33,NaN,301.926667


,PRICE,RV_PRICE
count,0.0,34367.000000
mean,NaN,156.289901
std,NaN,200.726414
min,NaN,7.930000
25%,NaN,75.354444
50%,NaN,110.046667
75%,NaN,169.241667
max,NaN,9999.990000


In [44]:
print(df.shape)
display(df.head(5))

(296291, 4392)


,ITEM_ID,SHP_WEIGHT,SHP_LENGTH,SHP_WIDTH,SHP_HEIGHT,PRICE,TITLE,LEN_ATR,SCORE,EXCEDIDO,...,DT_MODEL_X900,DT_MODEL_XBOX_360,DT_MODEL_XBOX_ONE,DT_MODEL_XBOX_ONE_S,DT_MODEL_XENON,DT_MODEL_XIAOMI,DT_MODEL_XTREME,DT_MODEL_X_PLAY,DT_MODEL__,RV_PRICE
0,SN7ISIGQ9J,235.0,25.0,25.0,10.0,68.0,Ácido Hidroquinona 20% 30g + Sabonete Pré Pe...,6,0.000326,0,...,0,0,0,0,0,0,0,0,0,68.0
1,JGEV50GW2U,1757.0,23.0,17.0,16.0,145.9,Inalador E Nebulizador Infantil Nebdog Superfl...,9,0.004556,0,...,0,0,0,0,0,0,0,0,0,145.9
2,JGEV50GW2U,1748.0,23.0,17.0,16.0,145.9,Inalador E Nebulizador Infantil Nebdog Superfl...,9,0.004556,0,...,0,0,0,0,0,0,0,0,0,145.9
3,JGEV50GW2U,1757.0,23.0,17.0,16.0,145.9,Inalador E Nebulizador Infantil Nebdog Superfl...,9,0.004556,0,...,0,0,0,0,0,0,0,0,0,145.9
4,TN0538QD0A,1780.0,23.0,17.0,16.0,145.9,"Inalador Infantil, 2 Anos De Garantia, G-tech,...",9,0.000224,0,...,0,0,0,0,0,0,0,0,0,145.9


In [45]:
df.dtypes

ITEM_ID                      object
SHP_WEIGHT                  float64
SHP_LENGTH                  float64
SHP_WIDTH                   float64
SHP_HEIGHT                  float64
                         ...       
DT_MODEL_XIAOMI    Sparse[uint8, 0]
DT_MODEL_XTREME    Sparse[uint8, 0]
DT_MODEL_X_PLAY    Sparse[uint8, 0]
DT_MODEL__         Sparse[uint8, 0]
RV_PRICE                    float64
Length: 4392, dtype: object

In [47]:
display(df.shape)
display(df_base.shape)

(296291, 4392)

(296291, 17)

In [54]:
#Agrego el nuevo precio calculado al dataset original
df_final=pd.concat([df_base, df.RV_PRICE], axis=1)

In [55]:
display(df_final.dtypes)
display(df_final.describe())

ITEM_ID          object
SHP_WEIGHT      float64
SHP_LENGTH      float64
SHP_WIDTH       float64
SHP_HEIGHT      float64
PRICE           float64
STATUS           object
TITLE            object
LEN_ATR           int64
DT_CAT_PROD      object
DT_CONDITION     object
DT_DOMAIN        object
DT_SELLER        object
DT_BRAND         object
DT_MODEL         object
SCORE           float64
EXCEDIDO          int64
RV_PRICE        float64
dtype: object

,SHP_WEIGHT,SHP_LENGTH,SHP_WIDTH,SHP_HEIGHT,PRICE,LEN_ATR,SCORE,EXCEDIDO,RV_PRICE
count,296291.000000,296291.000000,296291.000000,296291.000000,261924.000000,296291.000000,296291.000000,296291.000000,296291.000000
mean,1861.161446,31.426163,21.318484,11.562582,176.781718,7.918637,0.122348,0.044925,174.404858
std,3254.785995,18.619454,11.301392,8.358128,444.476057,5.676726,0.261096,0.207141,423.509403
min,1.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000014,0.000000,0.100000
25%,253.000000,20.000000,13.000000,5.000000,48.900000,4.000000,0.000282,0.000000,50.424444
50%,680.000000,25.000000,20.000000,10.000000,100.000000,8.000000,0.002085,0.000000,103.223333
75%,1948.000000,36.000000,25.000000,16.000000,179.000000,11.000000,0.055526,0.000000,178.000000
max,50000.000000,288.200000,165.000000,105.000000,30000.000000,60.000000,0.999856,1.000000,30000.000000


In [57]:
cols=['ITEM_ID', 'SHP_WEIGHT', 'SHP_LENGTH', 'SHP_WIDTH', 'SHP_HEIGHT',
       'RV_PRICE', 'STATUS', 'TITLE', 'LEN_ATR', 'DT_CAT_PROD', 'DT_CONDITION',
       'DT_DOMAIN', 'DT_SELLER', 'DT_BRAND', 'DT_MODEL', 'SCORE', 'EXCEDIDO']

In [59]:
df_final[cols].to_csv('dataset/meli_dataset_c.csv', index=False)